## New User Recruitment - Identifying Potential Customers at Risk

In this project, we want to identify first time experience users who will book within 2 days of interacting with the site and users who would not. Users who would not make a reservation within 2 days are at risk of chooising another competitor. We can also custom target users to improve Lead Velocity Rate.

Lead Velocity Rate is a SaaS metric defined to be how many users are you working on to converting to actual customers. 


This is important because, we want to convert these users who are at risk to actual customers. By correctly identifying which new users we are at risk of losing, companies like AirBnB can run personalized promotion (first user discounts) to convert these users to lower the risk of losing them to competitors.

Terminologies:
* User activation is when a user have started using the product, in our case - makes a booking on AirBnB.
* Lead Velocity Rate is the number of user you are working on to be converted to actual customers compared to previous month.
* User is someone who have interacted with the site.
* Potential Customers are users who are likely to be converted to be customers.

Data - We will use airbnb recruiting-new-user-bookings [dataset](https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings/data) from kaggle competition for this case study.

An important design of this project is to determine who are at risk. This will be our target variable. In this case study, we will define users who had their first activity and did not make a booking within 2 days or less. If users fits the condition, they are at risk of going to a competitor.

* Design Reasoning: We chose 2 days threshold because as business, because after two days the user are likely to turn to other competitors for alternative options.

#### Data
* train_users.csv - the training set of users
* test_users.csv - the test set of users
* id: user id
* date_account_created: the date of account creation
* timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
* date_first_booking: date of first booking
* gender
* age
* signup_method
* signup_flow: the page a user came to signup up from
* language: international language preference
* affiliate_channel: what kind of paid marketing
* affiliate_provider: where the marketing is e.g. google, craigslist, other
* first_affiliate_tracked: whats the first marketing the user interacted with before the signing up
* signup_app
* first_device_type
* first_browser
* country_destination: this is the target variable you are to predict

## EDA and Preprocessing

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('train_users_2.csv')
test_df = pd.read_csv('test_users.csv')

In [3]:
train_df.shape, test_df.shape

((213451, 16), (62096, 15))

#### Remove the country_destination column from train_df and combine both train and test datasets

In [4]:
booking = pd.concat([train_df.drop('country_destination',1), test_df])
booking.shape

(275547, 15)

In [5]:
booking.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome


#### We need to covnert the datetime columns to datetime objects

In [6]:
booking_time = booking.copy()
booking_time['date_account_created'] = pd.to_datetime(booking_time['date_account_created'].astype(str))
booking_time['timestamp_first_active'] = pd.to_datetime(booking_time['timestamp_first_active'].astype(str))
booking_time['date_first_booking'] = pd.to_datetime(booking_time['date_first_booking'].astype(str))

#### We want to create a column that calculates the time delta between account made and first activity

In [7]:
booking_time['time_delta_account_first'] = abs(booking_time['date_account_created'] - 
                                               booking_time['timestamp_first_active'])


We want to find all our users who made an account and had his/her first activity within 24 hours apart

In [8]:
booking_time['serious_customer'] = [1 if time.days == 0  else 0 for time in 
                                    booking_time['time_delta_account_first']]
print(booking_time.shape)

(275547, 17)


In [9]:
booking_filter = booking_time[booking_time['serious_customer'] == 1]
print(booking_filter.shape)

(275376, 17)


In [10]:
#calculate the time between they first browse the website and they first booked a room
booking_delta = booking_filter.copy()
booking_delta['time_delta_book_activity'] = abs(booking_delta['date_first_booking'] - booking_delta['timestamp_first_active'])

Let's create our target variable column 'at_risk' 
* 1 if the user is at risk of not booking before 5 days
* 0 if the user is at not at risk of booking before 5 days (the user will book within 5 days)

In [34]:
# User who book within 5 days or less are not at risk
booking_delta['user_at_risk'] = [0 if (time.days <= 2) else 1 for time in booking_delta['time_delta_book_activity']]

In [35]:
booking_delta.user_at_risk.value_counts()

1    228677
0     46699
Name: user_at_risk, dtype: int64

In [36]:
print(f'{booking_delta.user_at_risk.value_counts()[0]} users are not at risk')
print(f'{booking_delta.user_at_risk.value_counts()[1]} users are at risk')

46699 users are not at risk
228677 users are at risk


#### We can pickle this DataFrame for later

In [50]:
# Drop some columsn that will not be used
# cols_to_drop = ['id', 'date_account_created', 'timestamp_first_active','time_delta_account_first', 'serious_customer',
#                 'date_first_booking', 'time_delta_book_activity']
# booking_delta.drop(cols_to_drop, 1, inplace = True)

# booking_delta.to_pickle('booking.pkl')

In [51]:
# Read pickled file
booking = pd.read_pickle('booking.pkl')

In [52]:
booking.columns

Index(['gender', 'age', 'signup_method', 'signup_flow', 'language',
       'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
       'signup_app', 'first_device_type', 'first_browser', 'user_at_risk'],
      dtype='object')

In [66]:
booking.first_device_type.value_counts()

Mac Desktop           106241
Windows Desktop        86905
iPhone                 39792
iPad                   18027
Other/Unknown          11161
Android Phone           9456
Android Tablet          2096
Desktop (Other)         1507
SmartPhone (Other)       191
Name: first_device_type, dtype: int64

In [56]:
booking.gender.value_counts()

-unknown-    129450
FEMALE        77450
MALE          68144
OTHER           332
Name: gender, dtype: int64

In [64]:
booking.first_affiliate_tracked.value_counts()

untracked        143082
linked            62028
omg               54832
tracked-other      6652
product            2351
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64

In [63]:
booking.affiliate_provider.value_counts()

direct                 181146
google                  65936
other                   13020
facebook                 3993
bing                     3715
craigslist               3474
padmapper                 836
vast                      830
yahoo                     653
facebook-open-graph       564
gsp                       455
meetup                    357
email-marketing           270
naver                      66
baidu                      32
yandex                     18
wayn                        8
daum                        3
Name: affiliate_provider, dtype: int64

In [62]:
booking.affiliate_channel.value_counts()

direct           181446
sem-brand         36423
sem-non-brand     20072
seo               14355
other              9540
api                8155
content            4117
remarketing        1268
Name: affiliate_channel, dtype: int64

In [60]:
booking.language.value_counts()

en           265368
zh             2634
fr             1508
es             1173
ko             1116
de              977
it              633
ru              508
ja              345
pt              322
sv              176
nl              134
tr               92
pl               75
da               75
no               51
cs               49
el               30
th               28
hu               25
id               23
fi               20
ca                6
is                5
hr                2
-unknown-         1
Name: language, dtype: int64

In [59]:
booking.signup_flow.value_counts()

0     205970
25     29826
12     11232
3       8818
2       6875
23      6408
24      4322
1       1042
8        308
6        300
21       197
5         36
20        14
16        11
15        10
14         4
10         2
4          1
Name: signup_flow, dtype: int64

In [57]:
booking.signup_method.value_counts()

basic       198106
facebook     74809
google        2438
weibo           23
Name: signup_method, dtype: int64

In [55]:
booking.age.value_counts()

30.0      7705
28.0      7683
29.0      7643
31.0      7533
27.0      7458
32.0      7185
33.0      6786
26.0      6740
34.0      6171
25.0      6147
35.0      5943
36.0      5028
24.0      4569
37.0      4473
38.0      4080
39.0      3636
23.0      3515
40.0      3326
41.0      3092
42.0      2727
45.0      2665
44.0      2580
22.0      2506
43.0      2484
46.0      2316
19.0      2021
47.0      2000
48.0      1777
50.0      1678
51.0      1628
          ... 
1934.0       3
111.0        2
1938.0       2
1929.0       2
1944.0       2
1926.0       2
1923.0       2
1940.0       2
1936.0       2
1951.0       2
1920.0       2
1942.0       1
2008.0       1
2002.0       1
1945.0       1
150.0        1
1952.0       1
1939.0       1
2000.0       1
1995.0       1
1948.0       1
1930.0       1
1941.0       1
1922.0       1
1968.0       1
1954.0       1
1953.0       1
132.0        1
1937.0       1
112.0        1
Name: age, Length: 145, dtype: int64

In [54]:
booking.first_affiliate_tracked.value_counts()

untracked        143082
linked            62028
omg               54832
tracked-other      6652
product            2351
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64